In [1]:
#import pandas as pd
import numpy as np

In [2]:
table = ''
with open('p107_network.txt','r') as file:
    for line in file:
        table += line.rstrip().replace('-','1000') + ','    # 1000 as it's bigger than the maximum value in the original table
                                                            # to unite the separate item ','
# [:-1] to remove the last ','
table = np.array(table[:-1].split(',')).reshape(40,40).astype(int)  
table

array([[1000, 1000, 1000, ..., 1000, 1000,  774],
       [1000, 1000,  262, ...,  102,  636,  289],
       [1000,  262, 1000, ...,  168,  432,  833],
       ...,
       [1000,  102,  168, ..., 1000,  371,  680],
       [1000,  636,  432, ...,  371, 1000,  540],
       [ 774,  289,  833, ...,  680,  540, 1000]])

In [14]:
ex_table = np.array([[1000,16,12,21,1000,1000,1000],
                  [16,1000,1000,17,20,1000,1000],
                  [12,1000,1000,28,1000,31,1000],
                  [21,17,28,1000,18,19,23],
                  [1000,20,1000,18,1000,1000,11],
                  [1000,1000,31,19,1000,1000,27],
                  [1000,1000,1000,23,11,27,1000]])

In [ ]:
# get minmum values first
def first_step(table):
    # creating a new table in which we will put our new edges
    new_table = np.zeros(table.shape)
    
    # first let's get the position of the minmum value in each row(or column as you like)
    mins = np.argmin(table, axis=1)
    
    # let's then fill the new table with the mins
    for row,col in enumerate(mins):
        value = table[row,col]
        new_table[row,col] = new_table[col,row] = value
              
    return new_table,(new_table.shape[0]-1)-(len(np.unique(new_table))-1)

In [ ]:
# get the index of the second smallest number
def get_second(table,idx):
    return list(table[idx]).index(sorted(table[idx])[1])

#let's combine each only-two-connected vertices to the nearest graph
def second_step(oldTable,newTable): 
    taken   = [] #containg taken rows
    only_us = []
    for row in range(newTable.shape[0]):
        #get the number of connected vertices for each vertices
        values ,indices = np.unique(newTable[row],return_index=True) 
        
        if  len(values)-1 == 1:  # len()-1 because i don't care about '0'
            for v,i in zip(values,indices): # just to make sure that we don't choose '0'
                if (v != 0) and (len(np.unique(newTable[i]))-1 == 1): #making sure that the other vertex is also connected to it only
                    idx_r,idx_c = get_second(oldTable,row),get_second(oldTable,i)
                    value_r,value_c = oldTable[row,idx_r],oldTable[i,idx_c]
                    if value_r < value_c :
                        if not row in taken:
                            only_us.append((value_r,row,idx_r))
                            taken.append(row)
                    else:
                        if not i in taken:
                            only_us.append((value_c,i,idx_c))
                            taken.append(i)
                            
    #now let's add the edges that have the smallest weights to our new table
    for value, row, col in only_us:
        newTable[row,col] = newTable[col,row] = value
    
    return newTable,(newTable.shape[0]-1)-(len(np.unique(newTable))-1)

In [ ]:
ex_new_table,edges_left = first_step(ex_table)
edges_left

In [ ]:
second_step(ex_table.copy(),ex_new_table.copy())

Now let's contue to see what will happen for the other table

In [ ]:
new_table,edges_left = first_step(table)
edges_left

In [ ]:
new_table,edges_left = second_step(table.copy(),new_table.copy())
edges_left

In [ ]:
pd.DataFrame(new_table).to_csv("new_table.csv")

# Using [Prim's Algorithm: Minimal Spanning Tree](https://www.youtube.com/watch?v=YyLaRffCdk4&ab_channel=BarryBrown)

In [4]:
def get_min_edge(network,visited,min_edges):
    Min = (999999,(0,0))
    for v in visited:
        row   = network[v].copy()
        idx   = np.argmin(row)
        edge  = (row[idx],(v,idx))
        #finding the smallest edge in this row other than the chosen ones and the ones that leads to the visited vertices    
        while np.sum(row == 1000) < len(row):
            if  not(edge in min_edges) and not(idx in visited):
                break
            row[idx] = 1000
            idx   = np.argmin(row)
            edge  = (row[idx],(v,idx))
        #print('-----------')
        #print(edge)
        #Testing
        if edge[0] <= Min[0]:
            #print('True')
            Min = edge
    
    #appending the results
    visited.append(Min[1][1])
    min_edges.append(Min)
    
    return min_edges,visited

In [13]:
#Note: there may be two equal weights for two different edges
#that's why i can't use np.unique then sum()
def get_weight(network):
    taken  = [] 
    weight = 0
    for row in range(network.shape[0]):
        for col in range(network.shape[1]):
            if (not sorted([row,col]) in taken) and (network[row,col] != 1000):
                weight += network[row,col]
                #print(network[row,col])
            taken.append(sorted([row,col]))
    
    return weight

In [19]:
def get_sol(network):
    visited     = [0]                             #I'll begin with the first row 
    min_edges   = []
    # let's iterate (vertices-1) times
    for _ in range(network.shape[0]-1):      # '_' as i am not interested in using it 
        min_edges,visited = get_min_edge(network,visited,min_edges)
    
    overall_weight = get_weight(network)
    minmum_weight  = sum([min_edges[i][0] for i in range(len(min_edges))])
    #print(min_edges)
    print("Maximum saving for this network is: " + str(overall_weight-minmum_weight) )

In [20]:
%time get_sol(table)

Maximum saving for this network is: 259679
Wall time: 391 ms
